# NodeDynamicsDemo.jl

This module and Jupyter notebook aim to illustrate the differences between
explicit, implicit, and analytical solutions to a system of linear differential
equations depicting a chain of energy nodes connected via diffusion.
Before we get started, it's important to understand the concept of an
*energy node* for the purposes of this demonstration:

> Note that there are a plethora of methods for discretising linear differential equations. However, they are not discussed as they lose in simplicity to the Euler methods, and in accuracy to the analytical solution.

In [ ]:
# Here, we activate the relevant environment and load modules of interest.

using Pkg
Pkg.activate(@__DIR__)
using NodeDynamicsDemo

## Energy node

For the purposes of this demonstration, an *energy node* is defined entirely
by 5 properties:

1. `energyContent`: The current energy content on the node.
2. `selfDischarge`: The rate of self-discharge from the node relative to its `energy_content`.
3. `diffCoeffPrev`: The rate of energy diffusion from this node to the previous node in the chain.
4. `diffCoeffNext`: The rate of energy diffusion from this node to the next node in the chain.
5. `influx`: An optional influx of energy into the node at given times.

The exact units for the above properties don't matter for the purposes of this demonstration.
The first node in the chain is connected to a set `boundary_first`, and the last
node is connected to a set `boundary_last`. The boundaries can also be left empty,
in which case the first and last nodes are only connected to one node each.

> Note that in this demonstration, we're modelling a 1D chain of interlinked nodes. There's nothing preventing from connecting nodes arbitrarily, the math just becomes a tad more complicated.

### Node chain for testing

For testing that the equations work, we want to define a chain with an easily
deductible end state.
Here, let's define a chain that starts with all nodes at idential states,
with the boundaries fixed at zero, and no external influxes.
With self-discharge and diffusion both tending towards zero, we should see
the chain start "cooling" towards zero, with nodes towards the end of the chain
cooling faster due to the fixed boundary states.
Ultimately, all chains should reach stable zeroes across all nodes.

Furthermore, since the chain is symmetric, we should only see the lines for
the later nodes, as they overlap their earlier symmetric counterparts

In [ ]:
## Here, we define our node chain as a `Vector` of `EnergyNode`s.

# First, let's define the timesteps.
timesteps = ones(100)

# Next, let's define the nodes.
nodes = [
    EnergyNode(
        [10.0],
        0.05,
        0.05,
        0.05,
        zeros(length(timesteps))
    )
    for i in 1:5
]

# Set up the chain
chain = NodeChain(
    nodes,
    zeros(length(timesteps)),
    zeros(length(timesteps)),
    timesteps
)

## Discretising and solving the dynamics

For solving the dynamics of the energy node chain, we start with the differential
equation describing the energy balance on each node:
$$
\frac{dE_n(t)}{dt} = - \sigma_{n} E_n(t) + \rho_{n,n-1} \big( E_{n-1}(t) - E_n(t) \big) + \rho_{n,n+1} \big( E_n(t) - E_{n+1}(t) \big) + P_n(t)
$$
- $E_n(t)$ is energy content of node `n` at time `t`.
- $\sigma_{n}$ is the `selfDischarge` rate from node `n`.
- $\rho_{n,n-1}$ is the `diffCoeffPrev` of node `n`.
- $\rho_{n,n+1}$ is the `diffCoeffNext` of node `n`.
- $P_n(t)$ is the optional energy `influx` on node `n`.

The above differential equation serves as the basis for all the different methods detailed in the following sections.

> Note that the self-discharge and diffusion coefficients are assumed to be time-invariant for simplicity. For the simpler methods they don't really need to be, but can require a lot of matrix inversions if they cannot be assumed constant.

## Euler method

The simplest method to discretise and solve the root differential equation is the
[Euler Method](https://en.wikipedia.org/wiki/Euler_method), sometimes called
*foward Euler method* or *explicit Euler method*.
The method works simply by assuming that over a short enough period $\Delta t$,
the different time-varying parameters can be assumed to stay approximately constant.
Thus, the differential equation can be discretised and the energy content on the
next time step can be solved simply via:
$$
\frac{E_{n,t+\Delta t} - E_{n,t}}{\Delta t} = - \sigma_{n} E_{n,t} + \rho_{n,n-1} \big( E_{n-1,t} - E_{n,t} \big) + \rho_{n,n+1} \big( E_{n+1,t} - E_{n,t} \big) + P_{n,t} \\
\Leftrightarrow E_{n,t+\Delta t} = E_{n,t} + \big( - \sigma_{n} E_{n,t} + \rho_{n,n-1} \big( E_{n-1,t} - E_{n,t} \big) + \rho_{n,n+1} \big( E_{n+1,t} - E_{n,t} \big) + P_{n,t} \big) \Delta t
$$
> Note that solving the energy content on the next time step isn't really a concern for MILP models, since both of the above equations can easily be cast as linear constraints.

However, the key assumption here is *a short enough period $\Delta t$*. With dynamics too fast
for the used time step, this method is prone to numerical instabilities and oscillations.

In [ ]:
## Let's solve the explicit Euler dynamics for our energy chain.

forward_euler_chain = deepcopy(chain)
@time solve_explicit_dynamics!(forward_euler_chain)
fw_plt = plot_chain(forward_euler_chain)

## Backward Euler method

The next simplest method is the so-called
[Backward Euler method](https://en.wikipedia.org/wiki/Backward_Euler_method) or
*implicit Euler method*. Similar to its explicit counterpart, the method
assumes that time-varying stuff stays approximately constant over the time step.
However, instead of projecting the energy content of the node on the next time step,
the implicit method calculates the current energy content of the node accounting
for current changes, using the previous energy content only as a starting point:
$$
\frac{E_{n,t} - E_{n,t-\Delta t}}{\Delta t} = - \sigma_{n} E_{n,t} + \rho_{n,n-1} \big( E_{n-1,t} - E_{n,t} \big) + \rho_{n,n+1} \big( E_{n+1,t} - E_{n,t} \big) + P_{n,t} \\
$$
> Note that solving the energy content on the current time step isn't a concern for MILP models, as the above equation can be cast as a constraint as is

For an isolated node, the current energy content can be solved trivially.
However, for a system of nodes such as our chain, the whole system needs to be
solved in unison, as the current energy contents of the nodes depend on each other.
The problem can be cast as a system of linear equations:
$$
\underbrace{
    \begin{bmatrix}
    \frac{1}{\Delta t} + \sigma_0 + \rho_{0,-1} + \rho_{0,1} & - \rho_{0,1} & 0 & 0 & \dots & 0 & 0 & 0 \\
    - \rho_{1,0} & \frac{1}{\Delta t} + \sigma_1 + \rho_{1,0} + \rho_{1,2} & - \rho_{1,2} & 0 & \dots & 0 & 0 & 0 \\
    \dots & \dots & \dots & \dots & \dots & \dots & \dots & \dots \\
    0 & 0 & 0 & 0 & \dots & - \rho_{N-1,N-2} & \frac{1}{\Delta t} + \sigma_{N-1} + \rho_{N-1,N-2} + \rho_{N-1,N} & - \rho_{N-1,N} \\
    0 & 0 & 0 & 0 & \dots & 0 & - \rho_{N,N-1} & \frac{1}{\Delta t} + \sigma_N + \rho_{N,N-1} + \rho_{N,N+1} 
    \end{bmatrix}
}_{\underline{A}}
\begin{bmatrix}
E_{0,t} \\ E_{1,t} \\ E_{2,t} \\ \dots \\ E_{N-2,t} \\ E_{N-1,t} \\ E_{N,t}
\end{bmatrix}
=
\begin{bmatrix}
\frac{E_{0,t-\Delta t}}{\Delta t} + P_{0,t} + \rho_{-1,0} E_{-1,t} \\
\frac{E_{1,t-\Delta t}}{\Delta t} + P_{1,t} \\
\dots \\
\frac{E_{N-1,t-\Delta t}}{\Delta t} + P_{N-1,t} \\
\frac{E_{N,t-\Delta t}}{\Delta t} + P_{N,t} + \rho_{N+1,N} E_{N+1,t}
\end{bmatrix}
$$
> Note that $E_{-1,t}$ and $E_{N+1,t}$ represent the set boundary conditions, if any. If the boundaries are left free, the terms are simply omitted.

Assuming that we know or can set the initial state of our system,
the right-hand side on the initial time step $t-\Delta t$ is known.
Thus, we can solve the vector of current $t$ energy contents of the system
by inverting matrix $\underline{A}$ and multiplying both sides with its inverse from the left.

In [ ]:
## Let's solve the implicit dynamics:

backward_euler_chain = deepcopy(chain)
@time solve_implicit_dynamics!(backward_euler_chain)
bw_plt = plot_chain(backward_euler_chain)

## Analytical solution

With simplifying assumptions similar to those required by the Euler methods,
the root linear differential equation system can actually be solved analytically.
Casting the linear differential equation system in matrix form:
$$
\begin{bmatrix}
\dot{E}_0(t) \\
\dot{E}_1(t) \\
\dots \\
\dot{E}_{N-1}(t) \\
\dot{E}_{N}(t) \\
\end{bmatrix}
=
\underbrace{
    \begin{bmatrix}
    -\sigma_{0} -\rho_{0,-1} -\rho_{0,1} & \rho_{0,1} & 0 & \dots & 0 & 0 \\
    \rho_{1,0} & -\sigma_{1} -\rho_{1,0} - \rho_{1,2} & \rho_{1,2} & 0 & \dots & 0 \\
    \dots & \dots & \dots & \dots & \dots & \dots \\
    0 & \dots & 0 & \rho_{N-1,N-2} & -\sigma_{N-1} -\rho_{N-1,N-2} - \rho_{N-1,N} & \rho_{N-1,N} \\
    0 & 0 & \dots & 0 & \rho_{N,N-1} & -\sigma_{N} -\rho_{N,N-1} -\rho_{N,N+1}
    \end{bmatrix}
}_{B}
\begin{bmatrix}
E_0(t) \\
E_1(t) \\
\dots \\
E_{N-1}(t) \\
E_{N}(t) \\
\end{bmatrix}
+
\begin{bmatrix}
P_0(t) + \rho_{-1,0} E_{-1,t} \\
P_1(t) \\
\dots \\
P_{N-1}(t) \\
P_{N}(t) + \rho_{N+1,N} E_{N+1,t} \\
\end{bmatrix}
\\
\Leftrightarrow \underline{\dot{E}}(t) = \underline{B} \underline{E}(t) + \underline{P}(t)
$$
> Here, the dot indicates time derivative and the underlines indicate matrices and/or vectors.

It can be shown, that for an invertible matrix $\underline{B}$, all matrices
$\underline{B}$, $\underline{B}^{-1}$, and $e^{\underline{B}t}$ are mutually
commutative. Furthermore, the derivative of the matrix exponential function
can be shown to be $\frac{d}{dt}e^{\underline{B}t} = \underline{B}e^{\underline{B}t}$.
**As long as $\underline{B}$ is time-invariant**
*(or at the very least can be assumed to be locally time-invariant)*,
the linear differential equation
system much like one would can be solved similar to
a regular linear differential equation:
$$
\underline{\dot{E}}(t) = \underline{B} \underline{E}(t) + \underline{P}(t) \\
\Leftrightarrow \underline{\dot{E}}(t) - \underline{B} \underline{E}(t) = \underline{P}(t) \qquad \big|\, e^{-\underline{B}t} \times \\
\Leftrightarrow e^{-\underline{B}t} \underline{\dot{E}}(t) - \underline{B} e^{-\underline{B}t} \underline{E}(t) = e^{-\underline{B}t} \underline{P}(t) \\
\Leftrightarrow \frac{d}{dt}\big( e^{-\underline{B}t} \underline{E}(t) \big) = e^{-\underline{B}t} \underline{P}(t) \qquad \big|\, \int_{t}^{t+\Delta t}dt \\
\Leftrightarrow \big( e^{-\underline{B}t} \underline{E}(t) \big)\big|_{t}^{t+\Delta t} = \int_{t}^{t+\Delta t} e^{-\underline{B}t} \underline{P}(t) dt
$$
> In order to proceed, we'll need some information about the `influx` and its time dependency. E.g. assuming it is constant over the desired time step $\Delta t$.

$$
\Leftrightarrow \big( e^{-\underline{B}t} \underline{E}(t) \big)\big|_{t}^{t+\Delta t} = -\underline{B}^{-1} \big( e^{-\underline{B}t} \big)\big|_{t}^{t+\Delta t} \underline{P}_t \\
\Leftrightarrow e^{-\underline{B}(t + \Delta t)} \underline{E}(t + \Delta t) - e^{-\underline{B}t} \underline{E}(t) = -\underline{B}^{-1} \big( e^{-\underline{B}(t + \Delta t)} - e^{-\underline{B}(t)} \big) \underline{P}_t \qquad \big|\, e^{\underline{B}(t + \Delta t)} \times \\
\Leftrightarrow \underline{E}(t + \Delta t) = e^{\underline{B}\Delta t} \underline{E}(t) - (\underline{I} - e^{\underline{B}\Delta t})\underline{B}^{-1} \underline{P}_t
$$
allowing us to solve the energy content of the nodes at time $t + \Delta t$
based on the energy content at time $t$ and some fancy matrix math on the
coefficient matrix $\underline{B}$.

> Note that here we're talking about *time*, not *time steps*. The only "discretization" we need is some assumption on how $\underline{P}(t)$ behaves. Assuming it constant is not necessary, but greatly simplifies tha calculations.

In [ ]:
## Let's solve the analytical dynamics:

analytical_chain = deepcopy(chain)
@time solve_analytical_dynamics!(analytical_chain)
an_plt = plot_chain(analytical_chain)

## Comparing the solutions

Next, let's compare the solutions a bit.
From the example chain above, we probably won't see a whole lot of difference
between the solutions, as the time steps are short enough compared to the
self-discharge and diffusion parameters.
However, we should still see the analytical solution in between the explicit
and implicit solutions if we look close enough.

In [ ]:
## Let's overlay the plots from before to see the differences in the dynamics.

using Plots
colors = palette(:auto, length(chain.nodes))

comparison_plt = plot_chain(analytical_chain; palette=colors)
plot_chain!(comparison_plt, forward_euler_chain; linestyle=:dash, palette=colors)
plot_chain!(comparison_plt, backward_euler_chain; linestyle=:dashdotdot, palette=colors)

Ok, seems like it's behaving as expected, although it's a bit difficult to tell.
Let's define a different chain to see if we can get easier to see results.


### New chains for dynamics comparison

Let's define a 5-node chain starting from zero, with one end tied to zero and
the other to one. That way, we should see each node more easily.

In [ ]:
# Constant timesteps still.
timesteps = ones(100)

# Still five nodes, this time starting from zero.
nodes = [
    EnergyNode(
        [0.0],
        0.05,
        0.05,
        0.05,
        zeros(length(timesteps))
    )
    for i in 1:5
]

# Set up the chain
chain = NodeChain(
    nodes,
    zeros(length(timesteps)),
    ones(length(timesteps)),
    timesteps
)

# Copy chains and solve dynamics
analytical_chain, forward_chain, backwards_chain = deepcopy.([chain, chain, chain])
solve_analytical_dynamics!(analytical_chain)
solve_explicit_dynamics!(forward_chain)
solve_implicit_dynamics!(backwards_chain)

# Define color palette and plot results
using Plots
colors = palette(:auto, length(chain.nodes))
comparison_plot = plot_chain(analytical_chain; palette=colors)
plot_chain!(comparison_plot, forward_chain; linestyle=:dash, palette=colors)
plot_chain!(comparison_plot, backwards_chain; linestyle=:dashdotdot, palette=colors)

Seems fine. The time resolution is too easy for the discretizations, though,
as we can barely see any differences between the nodes.

### Let's mess with the time steps and see what happens!

In [ ]:
# Timestep increases by 1 every step.
timesteps = cumsum(ones(20))

# Still five nodes, this time starting from zero.
nodes = [
    EnergyNode(
        [0.0],
        0.05,
        0.05,
        0.05,
        zeros(length(timesteps))
    )
    for i in 1:5
]

# Set up the chain
chain = NodeChain(
    nodes,
    zeros(length(timesteps)),
    ones(length(timesteps)),
    timesteps
)

# Copy chains and solve dynamics
analytical_chain, forward_chain, backwards_chain = deepcopy.([chain, chain, chain])
solve_analytical_dynamics!(analytical_chain)
solve_explicit_dynamics!(forward_chain)
solve_implicit_dynamics!(backwards_chain)

# Define color palette and plot results
using Plots
colors = palette(:auto, length(chain.nodes))
comparison_plot = plot_chain(analytical_chain; palette=colors)
plot_chain!(comparison_plot, forward_chain; linestyle=:dash, palette=colors)
plot_chain!(comparison_plot, backwards_chain; linestyle=:dashdotdot, palette=colors)

Ok, seems like the explicit approximation starts to break down at long timesteps.
This is to be expected, as the forward Euler is notorious for becoming
numerically unstable unless the time step is short enough for the application.

### In fact, let's see how the methods behave when using different time steps lengths!

In [ ]:
# Lets define the same time period using different time step lengths!
dts = [1,2,5,10]
timesteps_dict = Dict(
    dt => float.(repeat([dt], div(100, dt)))
    for dt in [1,2,5,10]
)
markers = Dict(
    float(dt) => m
    for (dt, m) in zip(dts, [:none, :circle, :rect, :diamond])
) # Markers to make the time resolutions easier to distinguish.

# Three nodes starting from zero, but using different timesteps
nodes_dict = Dict(
    dt => [
        EnergyNode(
            [0.0],
            0.05,
            0.05,
            0.05,
            zeros(length(timesteps))
        )
        for i in 1:3
    ]
    for (dt, timesteps) in timesteps_dict
)

# Set up the chains with different time step lengths.
chains_dict = Dict(
    dt => NodeChain(
        nodes_dict[dt],
        zeros(length(timesteps)),
        ones(length(timesteps)),
        timesteps
    )
    for (dt, timesteps) in timesteps_dict
)

# Initialize plots and loop solve
using Plots
plt_dict = Dict(
    method => plot()
    for method in [
        solve_analytical_dynamics!,
        solve_implicit_dynamics!,
        solve_explicit_dynamics!
    ]
)

# Loop over the methods and chains with different time resolutions.
for (method, plt) in plt_dict
    for (dt, chai) in chains_dict
        cha = deepcopy(chai)
        colors = palette(:auto, length(cha.nodes))
        method(cha)
        plot_chain!(
            plt,
            cha;
            palette=colors,
            marker=markers[dt],
            markersize=2,
            title="$(method)",
            size=(1800,400)
        )
    end
end

# Gather plots
plot(values(plt_dict)..., layout=(1,3), )

In the above graphs, markers represent the following time step lengths:
- No marker represents the most accurate solution at step length 1.
- Circle represents the next best solution at step length 2.
- Rectangles are the next step at length 5.
- Diamonds are the last step at length 10.

As expected, the implicit solution begins underestimating the changes
as the timestep is increased, while the explicit method overestimates them
until going ham at longer step lengths. Meanwhile, the analytical solution
behaves as expected, with all of the solutions on the same trajectory regardless
of the time resolution. This is because the analytical solution represents
the real undiscretized behaviour of the system, only sampled at the desired
points in time.